In [ ]:
import pandas as pd
import numpy as np
import os
import re


import seaborn as sns
import matplotlib.pyplot as plt


from datetime import datetime
from pandas_profiling import ProfileReport


In [ ]:
# definitions 
datain_path = 'data/'

explorations_path = 'explorations/'
if not os.path.exists(explorations_path): 
    os.makedirs(explorations_path)

In [ ]:
datasets = [
    ('train', 'Train.xlsx'),
    ('test', 'Test.xlsx')  
]
datasets = pd.DataFrame(datasets, columns=['name', 'path']).set_index('name')

dataset_name = 'test'
dataset_path = datasets.loc[dataset_name].path


In [ ]:

data = pd.read_excel(os.path.join(datain_path, dataset_path))
data.head()

# Explorations

## profile report

profile = ProfileReport(
    data,
    title='Raw data',
    minimal=False, 
    correlations={
    "pearson": {"calculate": True},
    "spearman": {"calculate": False},
    "kendall": {"calculate": False},
    "phi_k": {"calculate": False},
    "cramers": {"calculate": False},
    }
)
profile.to_file(os.path.join(explorations_path, 'profile_data_raw.html'))


In [ ]:
# 
data.info()

In [ ]:
# init 
cols_to_drop = []
cols_to_onehot = []


In [ ]:
# extract gender from name?!
salutation = data.Name.str.split(' ', n=1, expand=True)[0]
if salutation.nunique() != 3: 
    raise ValueError('Unexpected levels of salutation')
    
print(salutation.value_counts())

#gender = ['male' if s == 'Mr.' else 'female' for s in salutation]
#data['gender'] = gender

male = [1 if s == 'Mr.' else 0 if s in ['Mrs.', 'Miss'] else np.nan for s in salutation]
data['male'] = male

if data.male.isna().sum() > 0: 
    raise Warning('NAs instroduced')
    
cols_to_drop.append('Name')


sns.countplot(data=data, x='male')
plt.show()


In [ ]:
# Compute age from Birthday

# clean whitespaces
data.Birthday = data.Birthday.str.replace(' ', '')
# define date format
dob_format = '%B%d,%Y'

# transform Birthday to datetime, catching the leap year error 

## helper fct to subtract one day from datetime if error occurs
def subone(obj):
    val = int(obj.group(0))
    return str(val-1)

## init and loop over dates
dob = []
error_log = []
for i, d in enumerate(data.Birthday): 
    try: 
        dob.append(datetime.strptime(d, dob_format).date())

    except ValueError as e: 
        if str(e) == 'day is out of range for month': 
            dt = datetime.strptime(re.sub('\d{1,2}', subone, d, count=1), dob_format).date()
            error_log.append((d, dt))
            dob.append(dt)
        else: 
            raise NotImplementedError('Do not know how to deal with that error!')
            dt = np.nan
            error_log.append((d, dt))
            dob.append(dt)
        
# add age column 
data['age'] = [np.floor((datetime.strptime('2048-12-31', '%Y-%m-%d').date() - d).days / 365.2425) for d in dob]

# inspect
sns.histplot(data, x='age')
plt.show()
print('Min age:' , min(data.age))

# drop date col 
cols_to_drop.append('Birthday')


In [ ]:
data.info()

In [ ]:
# 'Native Continent' to bin 
data['from_europe'] = [1 if a == 'Europe' else 0 for a in data['Native Continent']]
sns.countplot(data=data, x='from_europe')
plt.show()

cols_to_drop.append('Native Continent')

In [ ]:
# Marital Status
base_col = 'Marital Status'
target_col = 'marital_status'
data[base_col].value_counts()

mapping = {
    'Married':'Married',
    'Single':'Single',
    'Divorced':'Divorced',
    'Separated':'Separated',
    'Widow':'Widow',
    'Married - Spouse Missing':'Married',
    'Married - Spouse in the Army':'Married'
}
    
data[target_col] = data[base_col].map(mapping)

sns.countplot(data=data, x=target_col)
plt.show()

cols_to_drop.append(base_col)
cols_to_onehot.append(target_col)

In [ ]:
# Lives with


base_col = 'Lives with'
target_col = 'household'
print(data[base_col].value_counts())

mapping = {
    'Wife': 'Partner',
    'Other Family': 'Family',
    'Children': 'Children',
    'Alone': 'Alone',
    'Husband': 'Partner',
    'Other relatives': 'Family'
}

print(mapping)
    
data[target_col] = data[base_col].map(mapping)

sns.countplot(data=data, x=target_col)
plt.show()

cols_to_drop.append(base_col)
cols_to_onehot.append(target_col)


In [ ]:
# 'Base Area' to bin 
data['basearea_northbury'] = [1 if a == 'Northbury' else 0 for a in data['Base Area']]
sns.countplot(data=data, x='basearea_northbury')
plt.show()

cols_to_drop.append('Base Area')

In [ ]:
# Education Level 
base_col = 'Education Level'
target_col = 'education'

#print(data[base_col].value_counts())

edu_mapping = pd.read_excel(os.path.join(datain_path, 'edu_mapping.xlsx'), 'Tabelle2')
mapping_options = ['level_0', 'level_1', 'numeric']
m_option = mapping_options[0]

#mapping = dict(edu_mapping[['name', mapping_options[2]]].set_index('name'))
#mapping = {k:v for k,v in edu_mapping[['name', mapping_options[2]]].set_index('name').items()}
#mapping = edu_mapping[['name', mapping_options[2]]].set_index('name')
mapping = edu_mapping[['name', m_option]].rename(columns={m_option: target_col})

print(mapping)

data = data.merge(mapping, left_on=base_col, right_on='name', how='left')
data.drop(columns=['name'], inplace=True)  



sns.histplot(data=data, x=target_col)
plt.show()

cols_to_drop.append(base_col)
#cols_to_onehot.append(target_col)


data[[base_col, target_col]]

In [ ]:
# years of education 
base_col = 'Years of Education'
target_col = 'education_years'
data.rename(columns={base_col: target_col}, inplace=True)

data.head()
#sns.histplot(data=data, y=target_col)

In [ ]:
# Employment Sector
base_col = 'Employment Sector'
target_col = 'empl_sector'

print(data[base_col].value_counts())

mapping = {
    'Private Sector - Services ': 'private',
    'Self-Employed (Individual)': 'self',
    'Public Sector - Others': 'public',
    '?': 'unknown',
    'Private Sector - Others': 'private',
    'Self-Employed (Company)': 'self',
    'Public Sector - Government': 'public',
    'Unemployed': 'delete',
    'Never Worked': 'delete'
    }

print(mapping)
    
data[target_col] = data[base_col].map(mapping)

sns.countplot(data=data, x=target_col)
plt.show()

cols_to_drop.append(base_col)
cols_to_onehot.append(target_col)

In [ ]:
# role
base_col = 'Role'
target_col = 'empl_role'

mapping = {
    'Professor': 'Professor',
    'Management': 'Management',
    'Repair & constructions': 'blue_collor',
    'Administratives': 'Administratives',
    'Sales': 'Administratives',
    'Other services': 'Services',
    'Machine Operators & Inspectors': 'blue_collor',
    '?': 'unknown',
    'Transports': 'blue_collor',
    'Cleaners & Handlers': 'blue_collor',
    'Agriculture and Fishing': 'blue_collor',
    'IT': 'Administratives',
    'Security': 'blue_collor',
    'Household Services': 'blue_collor',
    'Army': 'blue_collor'
}

print(data[base_col].value_counts())

print(mapping)
    
data[target_col] = data[base_col].map(mapping)

sns.countplot(data=data, x=target_col)
plt.show()

cols_to_drop.append(base_col)
cols_to_onehot.append(target_col)

In [ ]:
# Working Hours per week
base_col = 'Working Hours per week'
target_col = 'working_hrs_week'

data.rename(columns={base_col: target_col}, inplace=True)

sns.histplot(data=data, x=target_col)
plt.show()

data.head()



In [ ]:
# Money Received
base_col = 'Money Received'
target_col = 'group_b_received_money'

data[target_col] = [1 if v != 0 else 0 for v in data[base_col]]

cols_to_drop.append(base_col)


sns.countplot(data=data, x=target_col)
plt.show()

data[[base_col, target_col]]



In [ ]:
# Ticket Price
base_col = 'Ticket Price'
target_col = 'group_c_payed'

data[target_col] = [1 if v != 0 else 0 for v in data[base_col]]

cols_to_drop.append(base_col)


sns.countplot(data=data, x=target_col)
plt.show()

data[[base_col, target_col]]

In [ ]:
# drop cols 
data.drop(columns=cols_to_drop, inplace=True)

In [ ]:
## profile report

profile = ProfileReport(
    data,
    title=f'Cleaned data {dataset_name}' ,
    minimal=False, 
    correlations={
    "pearson": {"calculate": True},
    "spearman": {"calculate": False},
    "kendall": {"calculate": False},
    "phi_k": {"calculate": False},
    "cramers": {"calculate": False},
    }
)
profile.to_file(os.path.join(explorations_path, f'profile_data_cleaned_{dataset_name}.html'))


# Explorations

# Feature Engineering ideas
- age + household: age diff to mean of hh group
- 

### Imputations: 
- empl_sector == unkown

In [ ]:


# one hot encode 
#pd.get_dummies(data=data, columns=cols_to_onehot)